Load and preprocess the datasets (MNIST, Fashion MNIST, and CIFAR-10):
python


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.utils import to_categorical

def load_and_preprocess_data(dataset_name):
    if dataset_name == 'mnist':
        (x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
    elif dataset_name == 'fashion_mnist':
        (x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()
    elif dataset_name == 'cifar10':
        (x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
    else:
        raise ValueError("Invalid dataset name. Choose from 'mnist', 'fashion_mnist', or 'cifar10'.")

    x_train = x_train.astype('float32') / 255.0
    x_test = x_test.astype('float32') / 255.0
    y_train = to_categorical(y_train, 10)
    y_test = to_categorical(y_test, 10)

    if len(x_train.shape) < 4:  # If the dataset is grayscale, convert to RGB
        x_train = np.repeat(x_train[..., np.newaxis], 3, axis=-1)
        x_test = np.repeat(x_test[..., np.newaxis], 3, axis=-1)

    return x_train, y_train, x_test, y_test

dataset_name = "mnist"  # Change this to 'fashion_mnist' or 'cifar10' for other datasets
x_train, y_train, x_test, y_test = load_and_preprocess_data(dataset_name)


11490434/11490434 [==============================] - 0s 0us/step


a) Use simple neural network as well as LeNet-5 - two architectures:

In [ ]:
def simple_nn(input_shape, num_classes):
    model = models.Sequential([
        layers.Flatten(input_shape=input_shape),
        layers.Dense(128, activation='relu'),
        layers.Dense(num_classes, activation='softmax')
    ])
    return model

def lenet_5(input_shape, num_classes):
    model = models.Sequential([
        layers.Conv2D(6, (5, 5), activation='relu', input_shape=input_shape),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(16, (5, 5), activation='relu'),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dense(120, activation='relu'),
        layers.Dense(84, activation='relu'),
        layers.Dense(num_classes, activation='softmax')
    ])
    return model

input_shape = x_train.shape[1:]
num_classes = 10

for model_architecture in [simple_nn, lenet_5]:
    model = model_architecture(input_shape, num_classes)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    model.fit(x_train, y_train, epochs=10, batch_size=128, validation_split=0.2)
    model_score = model.evaluate(x_test, y_test)
    print(f"{model_architecture.__name__} Test Loss:", model_score[0])
    print(f"{model_architecture.__name__} Test Accuracy:", model_score[1])


Epoch 1/10
375/375 [==============================] - 18s 43ms/step - loss: 0.3057 - accuracy: 0.9124 - val_loss: 0.1725 - val_accuracy: 0.9518
Epoch 2/10
375/375 [==============================] - 5s 14ms/step - loss: 0.1441 - accuracy: 0.9583 - val_loss: 0.1264 - val_accuracy: 0.9640
Epoch 3/10
375/375 [==============================] - 4s 12ms/step - loss: 0.1013 - accuracy: 0.9702 - val_loss: 0.1085 - val_accuracy: 0.9676
Epoch 4/10
375/375 [==============================] - 5s 14ms/step - loss: 0.0762 - accuracy: 0.9781 - val_loss: 0.0991 - val_accuracy: 0.9705
Epoch 5/10
375/375 [==============================] - 4s 11ms/step - loss: 0.0586 - accuracy: 0.9827 - val_loss: 0.0911 - val_accuracy: 0.9735
Epoch 6/10
375/375 [==============================] - 4s 11ms/step - loss: 0.0468 - accuracy: 0.9861 - val_loss: 0.0885 - val_accuracy: 0.9737
Epoch 7/10
375/375 [==============================] - 6s 15ms/step - loss: 0.0366 - accuracy: 0.9896 - val_loss: 0.0932 - val_accuracy: 0.972

In [ ]:
!pip install tensorflow-addons


b) Demonstrate pretrained model with transfer learning (both EfficientNet and BiT):



In [ ]:
# 1. Install necessary libraries
!pip install -U tensorflow
!pip install tensorflow_hub

# 2. Import libraries
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

# 3. Load and preprocess the MNIST dataset
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

# Preprocess the data by converting the grayscale images to 3-channel images
def preprocess_data(images, labels):
    images = np.expand_dims(images, axis=-1)
    images = images.astype("float32") / 255.0
    images = np.repeat(images, 3, axis=-1)  # Convert to 3-channel images
    images = tf.image.resize(images, (32, 32))
    labels = tf.keras.utils.to_categorical(labels, 10)
    return images, labels




x_train, y_train = preprocess_data(x_train, y_train)
x_test, y_test = preprocess_data(x_test, y_test)

# 4. Load pre-trained models (EfficientNet and BiT)
efficientnet_url = "https://tfhub.dev/tensorflow/efficientnet/b0/feature-vector/1"
bit_url = "https://tfhub.dev/google/bit/m-r50x1/1"

def create_model(model_url, input_shape, num_classes):
    model = tf.keras.Sequential([
        tf.keras.layers.InputLayer(input_shape=input_shape),
        hub.KerasLayer(model_url, trainable=True),
        Dense(num_classes, activation='softmax')
    ])
    return model
efficientnet_model = create_model(efficientnet_url, (32, 32, 3), 10)
bit_model = create_model(bit_url, (32, 32, 3), 10)


# 5. Fine-tune the models
efficientnet_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                           loss='categorical_crossentropy',
                           metrics=['accuracy'])

bit_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

print("Training EfficientNet...")
efficientnet_history = efficientnet_model.fit(x_train, y_train, batch_size=64, epochs=1, validation_split=0.1)

print("\nTraining BiT...")
bit_history = bit_model.fit(x_train, y_train, batch_size=64, epochs=1, validation_split=0.1)

# 6. Evaluate the models
efficientnet_score = efficientnet_model.evaluate(x_test, y_test, verbose=0)
print("\nEfficientNet Test loss:", efficientnet_score[0])
print("EfficientNet Test accuracy:", efficientnet_score[1])

bit_score = bit_model.evaluate(x_test, y_test, verbose=0)
print("\nBiT Test loss:", bit_score[0])
print("BiT Test accuracy:", bit_score[1])


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Training EfficientNet...
844/844 [==============================] - 648s 721ms/step - loss: 0.3150 - accuracy: 0.9477 - val_loss: 0.1963 - val_accuracy: 0.9835

Training BiT...
844/844 [==============================] - 4719s 6s/step - loss: 0.5021 - accuracy: 0.8803 - val_loss: 0.0634 - val_accuracy: 0.9838

EfficientNet Test loss: 0.194539874792099
EfficientNet Test accuracy: 0.9825000166893005

BiT Test loss: 0.05297866836190224
BiT Test accuracy: 0.9861999750137329


c) Demonstrate image augmentation:

In [ ]:
from tensorflow.keras.datasets import mnist
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import layers, models

(x_train, y_train), (x_test, y_test) = mnist.load_data()

datagen = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True
)

model = models.Sequential([
    layers.InputLayer(input_shape=(28, 28, 1)),
    layers.experimental.preprocessing.Rescaling(1./255),
    layers.Conv2D(32, (3, 3), activation='relu', padding='same'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

datagen.fit(x_train.reshape(-1, 28, 28, 1))

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

model.fit(datagen.flow(x_train.reshape(-1, 28, 28, 1), y_train, batch_size=32), epochs=1, validation_data=(x_test.reshape(-1, 28, 28, 1), y_test))


In [ ]:
import matplotlib.pyplot as plt

# Generate some augmented images
augmented_images = datagen.flow(x_train.reshape(-1, 28, 28, 1), y_train, batch_size=1)

# Display the first 9 augmented images
fig, axs = plt.subplots(3, 3, figsize=(8, 8))
axs = axs.ravel()
for i in range(9):
    image, label = augmented_images.next()
    axs[i].imshow(image[0], cmap=plt.cm.gray)
    axs[i].set_title(f"Label: {label.argmax()}")
    axs[i].axis('off')

plt.show()


d) demonstrate with state of art image models - mlp-mixer

In [ ]:
!pip install tensorflow-addons


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_addons as tfa

In [ ]:
# 1. Install necessary libraries
!pip install tensorflow-addons
!pip install -U tensorflow

# 2. Import libraries
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_addons as tfa

# 3. Load and preprocess the MNIST dataset
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)

y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

# 4. Implement the MLP-Mixer model
def mlp_block(units, dropout_rate=0.1):
    return keras.Sequential([
        layers.Dense(units, activation=tf.nn.gelu),
        layers.Dropout(dropout_rate),
    ])

def mixer_block(hidden_dim, num_patches, dropout_rate=0.1):
    return keras.Sequential([
        layers.LayerNormalization(),
        layers.Dense(num_patches, use_bias=False),
        layers.Activation(tf.nn.gelu),
        layers.Dropout(dropout_rate),
        layers.LayerNormalization(),
        layers.Dense(hidden_dim, use_bias=False),
        layers.Dropout(dropout_rate),
    ])

def mlp_mixer(input_shape, hidden_dim, num_blocks, num_patches, dropout_rate=0.1):
    inputs = keras.Input(shape=input_shape)
    x = layers.Reshape((-1, input_shape[-1]))(inputs)
    
    for _ in range(num_blocks):
        x = layers.Add()([x, mlp_block(hidden_dim)(mixer_block(hidden_dim, num_patches)(x))])

    x = layers.LayerNormalization()(x)
    x = layers.GlobalAveragePooling1D()(x)
    outputs = layers.Dense(10, activation='softmax')(x)

    return keras.Model(inputs=inputs, outputs=outputs)

# 5. Train the model
input_shape = (28, 28, 1)
hidden_dim = 64
num_blocks = 8
num_patches = 16

model = mlp_mixer(input_shape, hidden_dim, num_blocks, num_patches)
model.compile(optimizer=tfa.optimizers.AdamW(learning_rate=0.001, weight_decay=1e-4),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, y_train, batch_size=256, epochs=2, validation_split=0.1)

# 6. Evaluate the model
score = model.evaluate(x_test, y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])
